In [28]:
import feedparser
import re

# Returns title and dictionary of word counts for an RSS feed
def getwordcounts(url):
    # Parse the feed
    d=feedparser.parse(url)
    wc={}
    # Loop over all the entries
    for e in d.entries:
        if 'summary' in e: summary=e.summary
        else: summary=e.description
        # Extract a list of words
        words=getwords(e.title+' '+summary)
        for word in words:
            wc.setdefault(word,0)
            wc[word]+=1
    return getattr(d.feed, 'title', 'Unknown title'), wc

In [29]:
def getwords(html):
    # Remove all the HTML tags
    txt=re.compile(r'<[^>]+>').sub('',html)
    # Split words by all non-alpha characters
    words=re.compile(r'[^A-Z^a-z]+').split(txt)
    # Convert to lowercase
    return [word.lower( ) for word in words if word!='']

In [30]:
wordlist = []
for word,wc in apcount.items():
    if ((wc/len(wordcounts) > 0.1) and (wc/len(wordcounts) < 0.4)):
        wordlist.append(word)

In [36]:
out=open('outputs/blogdata.txt','w')
out.write('Blog')
for word in wordlist: out.write('\t%s' % word) 
out.write('\n')
for blog,wc in wordcounts.items():
    out.write(blog)
    for word in wordlist:
        if word in wc: 
            out.write('\t%d' % wc[word])
        else: out.write('\t0')
    out.write('\n')